# Wide & Deep Learning for Recommender System

- Google에서 App Store를 활용해서 발표한 논문([링크](https://arxiv.org/pdf/1606.07792.pdf))


## Google 공식 문서
- Google의 AI Blog([링크](https://ai.googleblog.com/2016/06/wide-deep-learning-better-together-with.html))
- Google의 Tensorflow github([링크](https://github.com/tensorflow/tensorflow/blob/v2.4.0/tensorflow/python/keras/premade/wide_deep.py#L34-L219))
- TensorFlow v2.4 API
  - [tf.keras.experimental.WideDeepModel](https://www.tensorflow.org/api_docs/python/tf/keras/experimental/WideDeepModel?hl=en#methods_2)
  - [tf.estimator.DNNLinearCombinedClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNLinearCombinedClassifier)

## 함께볼만한 PyTorch Library
- [pytorch-widedeep](https://github.com/jrzaurin/pytorch-widedeep)

In [23]:
# !pip install pytorch-widedeep

In [24]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## DataLoader

In [25]:
data_path = "../04-Recommender-System-with-DeepLearning/kmrd/"
# %cd $data_path

# if not os.path.exists(data_path):
#   !git clone https://github.com/lovit/kmrd
#   !python setup.py install
# else:
#   print("data and path already exists!")

path = data_path + '/kmr_dataset/datafile/kmrd-small'

In [26]:
df = pd.read_csv(os.path.join(path,'rates.csv'))
train_df, val_df = train_test_split(df, test_size=0.2, random_state=1234, shuffle=True)



In [27]:
train_df.shape

(112568, 4)

In [28]:
train_df = train_df[:1000]

In [29]:
# Load all related dataframe
movies_df = pd.read_csv(os.path.join(path, 'movies.txt'), sep='\t', encoding='utf-8')
movies_df = movies_df.set_index('movie')

castings_df = pd.read_csv(os.path.join(path, 'castings.csv'), encoding='utf-8')
countries_df = pd.read_csv(os.path.join(path, 'countries.csv'), encoding='utf-8')
genres_df = pd.read_csv(os.path.join(path, 'genres.csv'), encoding='utf-8')

# Get genre information
genres = [(list(set(x['movie'].values))[0], '/'.join(x['genre'].values)) for index, x in genres_df.groupby('movie')]
combined_genres_df = pd.DataFrame(data=genres, columns=['movie', 'genres'])
combined_genres_df = combined_genres_df.set_index('movie')

# Get castings information
castings = [(list(set(x['movie'].values))[0], x['people'].values) for index, x in castings_df.groupby('movie')]
combined_castings_df = pd.DataFrame(data=castings, columns=['movie','people'])
combined_castings_df = combined_castings_df.set_index('movie')

# Get countries for movie information
countries = [(list(set(x['movie'].values))[0], ','.join(x['country'].values)) for index, x in countries_df.groupby('movie')]
combined_countries_df = pd.DataFrame(data=countries, columns=['movie', 'country'])
combined_countries_df = combined_countries_df.set_index('movie')

movies_df = pd.concat([movies_df, combined_genres_df, combined_castings_df, combined_countries_df], axis=1)

print(movies_df.shape)
print(movies_df.head())

(999, 7)
                      title                           title_eng    year   
movie                                                                     
10001                시네마 천국              Cinema Paradiso , 1988  2013.0  \
10002              빽 투 더 퓨쳐           Back To The Future , 1985  2015.0   
10003            빽 투 더 퓨쳐 2    Back To The Future Part 2 , 1989  2015.0   
10004            빽 투 더 퓨쳐 3  Back To The Future Part III , 1990  1990.0   
10005  스타워즈 에피소드 4 - 새로운 희망                    Star Wars , 1977  1997.0   

         grade         genres   
movie                           
10001   전체 관람가     드라마/멜로/로맨스  \
10002  12세 관람가         SF/코미디   
10003  12세 관람가         SF/코미디   
10004   전체 관람가  서부/SF/판타지/코미디   
10005       PG   판타지/모험/SF/액션   

                                                  people   country  
movie                                                               
10001  [4374, 178, 3241, 47952, 47953, 19538, 18991, ...  이탈리아,프랑스  
10002    [1076, 4603, 917,

In [30]:
movies_df.columns

Index(['title', 'title_eng', 'year', 'grade', 'genres', 'people', 'country'], dtype='object')

In [31]:
dummy_genres_df = movies_df['genres'].str.get_dummies(sep='/')
train_genres_df = train_df['movie'].apply(lambda x: dummy_genres_df.loc[x])
train_genres_df.head()

,SF,가족,공포,느와르,다큐멘터리,드라마,로맨스,멜로,모험,뮤지컬,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
92868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
94390,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22289,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
80155,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [32]:
dummy_grade_df = pd.get_dummies(movies_df['grade'], prefix='grade')
train_grade_df = train_df['movie'].apply(lambda x: dummy_grade_df.loc[x])
train_grade_df.head()

,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,grade_PG-13,grade_R,grade_전체 관람가,grade_청소년 관람불가
137023,True,False,False,False,False,False,False,False,False
92868,False,False,False,False,True,False,False,False,False
94390,True,False,False,False,False,False,False,False,False
22289,False,False,False,False,False,False,False,True,False
80155,True,False,False,False,False,False,False,False,False


In [33]:
train_df['year'] = train_df.apply(lambda x: movies_df.loc[x['movie']]['year'], axis=1)

In [34]:
train_df = pd.concat([train_df, train_grade_df, train_genres_df], axis=1)
train_df.head()

,user,movie,rate,time,year,grade_12세 관람가,grade_15세 관람가,grade_G,grade_NR,grade_PG,...,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
137023,48423,10764,10,1212241560,1987.0,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
92868,17307,10170,10,1122185220,1985.0,False,False,False,False,True,...,0,0,0,0,0,1,0,0,0,0
94390,18180,10048,10,1573403460,2016.0,True,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
22289,1498,10001,9,1432684500,2013.0,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0
80155,12541,10022,10,1370458140,1980.0,True,False,False,False,False,...,0,0,0,0,0,1,0,0,0,0


In [35]:
wide_cols = list(dummy_genres_df.columns) + list(dummy_grade_df.columns)
wide_cols

['SF',
 '가족',
 '공포',
 '느와르',
 '다큐멘터리',
 '드라마',
 '로맨스',
 '멜로',
 '모험',
 '뮤지컬',
 '미스터리',
 '범죄',
 '서부',
 '서사',
 '스릴러',
 '애니메이션',
 '액션',
 '에로',
 '전쟁',
 '코미디',
 '판타지',
 'grade_12세 관람가',
 'grade_15세 관람가',
 'grade_G',
 'grade_NR',
 'grade_PG',
 'grade_PG-13',
 'grade_R',
 'grade_전체 관람가',
 'grade_청소년 관람불가']

In [36]:
print(len(wide_cols))
print(wide_cols)

wide_cols = wide_cols[:3]

30
['SF', '가족', '공포', '느와르', '다큐멘터리', '드라마', '로맨스', '멜로', '모험', '뮤지컬', '미스터리', '범죄', '서부', '서사', '스릴러', '애니메이션', '액션', '에로', '전쟁', '코미디', '판타지', 'grade_12세 관람가', 'grade_15세 관람가', 'grade_G', 'grade_NR', 'grade_PG', 'grade_PG-13', 'grade_R', 'grade_전체 관람가', 'grade_청소년 관람불가']


In [37]:
# wide_cols = ['genre', 'grade']
# cross_cols = [('genre', 'grade')]
wide_cols

['SF', '가족', '공포']

In [38]:
import itertools
from itertools import product  
unique_combinations = list(list(zip(wide_cols, element)) 
                           for element in product(wide_cols, repeat = len(wide_cols))) 

print(unique_combinations)
cross_cols = [item for sublist in unique_combinations for item in sublist]
cross_cols = [x for x in cross_cols if x[0] != x[1]]
cross_cols = list(set(cross_cols))
print(cross_cols)

[[('SF', 'SF'), ('가족', 'SF'), ('공포', 'SF')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '가족')], [('SF', 'SF'), ('가족', 'SF'), ('공포', '공포')], [('SF', 'SF'), ('가족', '가족'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '가족'), ('공포', '가족')], [('SF', 'SF'), ('가족', '가족'), ('공포', '공포')], [('SF', 'SF'), ('가족', '공포'), ('공포', 'SF')], [('SF', 'SF'), ('가족', '공포'), ('공포', '가족')], [('SF', 'SF'), ('가족', '공포'), ('공포', '공포')], [('SF', '가족'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '가족'), ('가족', 'SF'), ('공포', '가족')], [('SF', '가족'), ('가족', 'SF'), ('공포', '공포')], [('SF', '가족'), ('가족', '가족'), ('공포', 'SF')], [('SF', '가족'), ('가족', '가족'), ('공포', '가족')], [('SF', '가족'), ('가족', '가족'), ('공포', '공포')], [('SF', '가족'), ('가족', '공포'), ('공포', 'SF')], [('SF', '가족'), ('가족', '공포'), ('공포', '가족')], [('SF', '가족'), ('가족', '공포'), ('공포', '공포')], [('SF', '공포'), ('가족', 'SF'), ('공포', 'SF')], [('SF', '공포'), ('가족', 'SF'), ('공포', '가족')], [('SF', '공포'), ('가족', 'SF'), ('공포', '공포')], [('SF', '공포'), ('가족', '가족'), ('공포', 'SF')], [('SF', '공포'), ('가족', '가족'), ('

In [39]:
# embed_cols = [('genre', 16),('grade', 16)]
embed_cols = list(set([(x[0], 16) for x in cross_cols]))
continuous_cols = ['year']

print(embed_cols)
print(continuous_cols)

[('공포', 16), ('가족', 16), ('SF', 16)]
['year']


In [40]:
target = train_df['rate'].apply(lambda x: 1 if x > 9 else 0).values

## Wide & Deep

In [59]:
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy
from pytorch_widedeep import Trainer

### Wide Component

In [44]:
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=cross_cols)
X_wide = preprocess_wide.fit_transform(train_df)
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [45]:
X_wide.size

9000

In [46]:
wide

Wide(
  (wide_linear): Embedding(29, 1, padding_idx=0)
)

### Deep Component

In [64]:
preprocess_tap = TabPreprocessor(embed_cols=embed_cols, continuous_cols=continuous_cols)
X_tab = preprocess_tap.fit_transform(train_df)
tabmlp = TabMlp(
    column_idx=preprocess_deep.column_idx,
    cat_embed_input=preprocess_deep.cat_embed_input,
    continuous_cols=continuous_cols,
)

In [53]:
tabmlp

TabMlp(
  (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
    (cat_embed): DiffSizeCatEmbeddings(
      (embed_layers): ModuleDict(
        (emb_layer_공포): Embedding(3, 16, padding_idx=0)
        (emb_layer_가족): Embedding(3, 16, padding_idx=0)
        (emb_layer_SF): Embedding(3, 16, padding_idx=0)
      )
      (embedding_dropout): Dropout(p=0.1, inplace=False)
    )
    (cont_norm): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (encoder): MLP(
    (mlp): Sequential(
      (dense_layer_0): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=49, out_features=200, bias=True)
        (2): ReLU(inplace=True)
      )
      (dense_layer_1): Sequential(
        (0): Dropout(p=0.1, inplace=False)
        (1): Linear(in_features=200, out_features=100, bias=True)
        (2): ReLU(inplace=True)
      )
    )
  )
)

### Build and Train

In [65]:
model = WideDeep(wide=wide, deeptabular=tabmlp)

# train and validate
trainer = Trainer(model, objective="binary", metrics=[Accuracy])
trainer.fit(
    X_wide=X_wide,
    X_tab=X_tab,
    target=target,
    n_epochs=5,
    batch_size=256,
)

  0%|                                                                                            | 0/4 [02:15<?, ?it/s]


KeyboardInterrupt: 